# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [73]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [74]:
output_data_file = "../output_data/cities.csv"
cities_df = pd.read_csv(output_data_file)
cities_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,44.60,57,0,8.05,CL,1614562578
1,Mataura,-46.1927,168.8643,72.00,72,91,1.99,NZ,1614562578
2,Ossora,59.2353,163.0719,-0.26,93,100,14.90,RU,1614562579
3,Ushuaia,-54.8000,-68.3000,44.60,65,20,9.22,AR,1614562579
4,Hilo,19.7297,-155.0900,75.20,69,90,8.05,US,1614562579
...,...,...,...,...,...,...,...,...,...
546,Port Augusta,-32.5000,137.7667,87.01,49,3,16.11,AU,1614563206
547,Noumea,-22.2763,166.4572,88.00,66,20,10.36,NC,1614563207
548,Novyy Urengoy,66.0833,76.6333,-21.57,87,92,10.38,RU,1614563208
549,Pontes E Lacerda,-15.2261,-59.3353,74.50,79,1,1.92,BR,1614563209


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [75]:
gmaps.configure(g_key)

In [77]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lon"]]
humidity = cities_df["Humidity"]

# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [78]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

hotel_df = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] > 70) & (cities_df['Wind Speed'] < 10) & (cities_df['Cloudiness']==0),:]
hotel_df = hotel_df.dropna()
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,Saint-Philippe,-21.3585,55.7679,75.20,69,0,5.75,RE,1614562603
31,Paungde,18.4833,95.5000,76.95,44,0,4.43,MM,1614562609
140,Yaan,7.3833,8.5667,75.72,80,0,6.46,NG,1614562701
201,Sakhon Nakhon,17.1612,104.1472,77.00,83,0,5.75,TH,1614562749
248,Kalmunai,7.4167,81.8167,75.87,71,0,2.28,LK,1614562787
260,Batticaloa,7.7102,81.6924,75.13,75,0,4.36,LK,1614562822
276,Salalah,17.0151,54.0924,73.40,60,0,2.30,OM,1614562829
308,Ler,8.3018,30.1418,76.68,17,0,7.74,SS,1614562863
312,Maiduguri,11.8464,13.1603,71.46,29,0,9.06,NG,1614562864
317,Tura,25.5198,90.2201,72.93,50,0,3.56,IN,1614562847


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [79]:
#Add empty Hotel Name column
hotel_df["Hotel Name"] = ""

# hotel_df
radius =5000
place_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
geo_url ="https://maps.googleapis.com/maps/api/geocode/json?"

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lon']
    try:
# #Set parameters
        place_params = {"key": g_key,"location": f"{lat},{lon}", "radius": radius,"type": "lodging"}

# # run a request using our params dictionary
        place_response = requests.get(place_url, params=place_params)

# # convert response to json
        hotel_name = place_response.json()

# # print(json.dumps(hotel_data, indent=4, sort_keys=True))

# # print(hotel_data["results"][1]["vicinity"])
        hotel_df.loc[index,"Hotel Name"]=hotel_name["results"][0]["name"].title()
    except:
        hotel_df.loc[index,"Hotel Name"]="Sorry! No Hotel"
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
12,Saint-Philippe,-21.3585,55.7679,75.20,69,0,5.75,RE,1614562603,"Chambres D'Hôte ""La Trinité"""
31,Paungde,18.4833,95.5000,76.95,44,0,4.43,MM,1614562609,Shwe Eain Matt Highly Cool Guest House
140,Yaan,7.3833,8.5667,75.72,80,0,6.46,NG,1614562701,Sorry! No Hotel
201,Sakhon Nakhon,17.1612,104.1472,77.00,83,0,5.75,TH,1614562749,บริษัท สกลแกรนด์พาเลซ จำกัด
248,Kalmunai,7.4167,81.8167,75.87,71,0,2.28,LK,1614562787,Vs Villa
260,Batticaloa,7.7102,81.6924,75.13,75,0,4.36,LK,1614562822,Grand Star Guest House
276,Salalah,17.0151,54.0924,73.40,60,0,2.30,OM,1614562829,Hamdan Plaza Hotel Salalah
308,Ler,8.3018,30.1418,76.68,17,0,7.74,SS,1614562863,Sorry! No Hotel
312,Maiduguri,11.8464,13.1603,71.46,29,0,9.06,NG,1614562864,Beach Guest Home
317,Tura,25.5198,90.2201,72.93,50,0,3.56,IN,1614562847,Hotel Rikman Continental


In [80]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [81]:
# Add marker layer ontop of heat map
marker_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(255, 0, 0, 1)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=3,
    info_box_content=hotel_info,
    hover_text="Click for info"
)


# Display figure
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))